# Cleaning the Data

The goal here is to get our data in a format that we can use to make predictions. We want the target variable to be the outcome of the election in each state (percent that democrats/republicans won by), and to then have columns for other factors of interest (generic ballot, presidential approval, characteristics of the state, etc). 

In [212]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as  np
pd.options.display.max_columns = None
%matplotlib inline
import datetime as dt

## Senate Election Results 

Senate election data is taken from the MIT-Harvard Elections Data Science Lab. It can be accessed [here](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/PEJ5QU)

In [213]:
senate_elections = pd.read_csv('1976-2020-senate.csv', encoding = "ISO-8859-1")

In [214]:
senate_elections.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party_detailed,writein,mode,candidatevotes,totalvotes,unofficial,version,party_simplified
0,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,SAM STEIGER,REPUBLICAN,False,total,321236,741210,False,20210114,REPUBLICAN
1,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,WM. MATHEWS FEIGHAN,INDEPENDENT,False,total,1565,741210,False,20210114,OTHER
2,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,DENNIS DECONCINI,DEMOCRAT,False,total,400334,741210,False,20210114,DEMOCRAT
3,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,ALLAN NORWITZ,LIBERTARIAN,False,total,7310,741210,False,20210114,LIBERTARIAN
4,1976,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,BOB FIELD,INDEPENDENT,False,total,10765,741210,False,20210114,OTHER


In [215]:
#We will start by examining election data starting in 2008 as it the most recent. To keep our data simple and 
#consistent, we will also ignore special elections. 
small_results = senate_elections[(senate_elections['special'] == False) & (senate_elections['year'] >= 2002)]
#California primaries result in the top two candidates advancing, regardless of party affiliation, so
#two democrats often run against each other in the general. Consequently, we will drop california from our model. 
small_results = small_results[ small_results['state'] != 'CALIFORNIA']

In [216]:
def get_single_year(year):
    ''' Creates a dataframe from a single year in small_results with two columns:
            state: the election state
            partisan_score: the democratic margin (percent). Negative values mean the republican won. 
            
        args: 
            year (int): the election year. 
    '''
    year_data = small_results[small_results['year'] == year].copy()
    year_data['perc'] = year_data['candidatevotes'] / year_data['totalvotes']
    year_data = year_data[(year_data['perc'] > .2) & (year_data['perc'] < .8)]
    year_data['party_simplified'] = year_data['party_simplified'].str.replace('OTHER', 'DEMOCRAT')
    year_data = year_data[['state','party_simplified','perc']].pivot_table(index='state', columns=['party_simplified'])
    year_data = year_data.reset_index(level=0)
    if len(year_data.columns) == 4:
        year_data.columns = ['state','democrat','other','republican']
    if len(year_data.columns) == 3:
        year_data.columns = ['state','democrat','republican']    
    year_data['partisan_score_{}'.format(year)] = year_data['democrat'] - year_data['republican']
    return year_data[['state','partisan_score_{}'.format(year)]]

In [217]:
def create_3_past(year):
    '''Creates a dataframe with the partisan score of the current year and the past three for each state. 
    '''
    current_year = get_single_year(year)
    two_ago = get_single_year(year - 2)
    four_ago = get_single_year(year - 4)
    six_ago = get_single_year(year - 6)
    current_year = current_year.merge(two_ago, how='outer', on='state')
    current_year = current_year.merge(four_ago, how='outer', on='state')
    current_year = current_year.merge(six_ago, how='outer', on='state')
    current_year.columns = ['state', 'partisan_score', 'two_ago_score', 'four_ago_score', 'six_ago_score']
    return current_year

In [218]:
def average_3_past(year):
    ''' Averages partisan scores of past three years and returns a new dataframe. 
    '''
    df = create_3_past(year)
    df['old_score_avg'] = df[['two_ago_score', 'four_ago_score', 'six_ago_score']].mean(axis=1)
    df = df[['state','partisan_score','old_score_avg']]
    df.dropna(inplace=True)
    df['year'] = year
    return df[['state','year','partisan_score','old_score_avg']]

In [219]:
average_3_past(2020)

,state,year,partisan_score,old_score_avg
0,ALABAMA,2020,-0.203587,-0.280915
1,ALASKA,2020,-0.127032,-0.021296
3,COLORADO,2020,0.093214,0.018603
4,DELAWARE,2020,0.215405,0.178679
5,GEORGIA,2020,-0.017790,-0.107166
6,IDAHO,2020,-0.293759,-0.345323
7,ILLINOIS,2020,0.160676,0.129675
8,IOWA,2020,-0.064782,-0.163868
9,KANSAS,2020,-0.114371,-0.202779
10,KENTUCKY,2020,-0.195338,-0.150080


## Generic Ballot

For the national generic ballot, we will use the average of generic ballot polling for the few weeks preceding the election. The data can be accessed on the Real Clear Politics website [here](https://www.realclearpolitics.com/epolls/other/2020_generic_congressional_vote-6722.html#polls) and on the Center for Politics website [here](https://centerforpolitics.org/crystalball/articles/the-key-to-forecasting-midterms-the-generic-ballot/). 

In [220]:
generic_ballot_avgs = {
    2020: .068,
    2018: .073,
    2016: .006,
    2014: -.024,
    2012: -.002,
    2010: -.1,
    2008: .1,
    2006: .12,
    2004: 0
}

In [221]:
def add_generic_ballot(year):
    df = average_3_past(year)
    df['generic_ballot'] = generic_ballot_avgs[year]
    return df

## Presidential Approval Score

For presidential approval polilng, we will use data from University of California Santa Barbara's American Presidency Project. The data all comes from Gallup polling, and is available [here](https://www.presidency.ucsb.edu/statistics/data).

In [222]:
trump = pd.read_excel('American Presidency Project - Approval Ratings for POTUS.xlsx',sheet_name='Donald Trump')
trump['president'] = 'trump'
obama = pd.read_excel('American Presidency Project - Approval Ratings for POTUS.xlsx', sheet_name='Barack Obama')
obama['president'] = 'obama'
bush = pd.read_excel('American Presidency Project - Approval Ratings for POTUS.xlsx', sheet_name='George W. Bush')
bush['president'] = 'bush'

In [223]:
pres_approval = pd.concat([trump, obama, bush])

In [224]:
pres_approval['pres_score'] = pres_approval['Approving'] - pres_approval['Disapproving']

In [225]:
pres_approval

,Start Date,End Date,Approving,Disapproving,Unsure/NoData,president,pres_score
0,2021-01-04,2021-01-15,34,62,4,trump,-28
1,2020-12-01,2020-12-17,39,57,4,trump,-18
2,2020-11-05,2020-11-19,43,55,2,trump,-12
3,2020-10-16,2020-10-27,46,52,2,trump,-6
4,2020-09-30,2020-10-15,43,55,2,trump,-12
...,...,...,...,...,...,...,...
277,2001-03-09,2001-03-11,58,29,13,bush,29
278,2001-03-05,2001-03-07,63,22,15,bush,41
279,2001-02-19,2001-02-21,62,21,17,bush,41
280,2001-02-09,2001-02-11,57,25,18,bush,32


In [226]:
def get_oct_nov_polls(year):
    return pres_approval[ (pres_approval['End Date'] > dt.datetime(year, 10, 1)) & ( pres_approval['End Date'] < dt.datetime(year, 11, 15))]

In [227]:
get_oct_nov_polls(2014)

,Start Date,End Date,Approving,Disapproving,Unsure/NoData,president,pres_score
297,2014-09-29,2014-10-05,43,53,4,obama,-10
298,2014-10-06,2014-10-12,41,55,4,obama,-14
299,2014-10-13,2014-10-19,41,54,5,obama,-13
300,2014-10-20,2014-10-26,42,53,5,obama,-11
301,2014-10-27,2014-11-02,42,53,5,obama,-11
302,2014-11-03,2014-11-09,40,54,6,obama,-14


In [228]:
def add_pres_approval(year):
    df = add_generic_ballot(year)
    polls = get_oct_nov_polls(year)
    df['pres_approval'] = polls['pres_score'].mean() * .01
    df['dem_pres'] = 0
    if year in [2010, 2012, 2014, 2016]:
        df['dem_pres'] = 1
    df['pres_approval_int'] = df['pres_approval'] * df['dem_pres']
    return df

## State Unemployment Rates

For state unemployment rates, we will use data from the Bureau of Labor Statistics. It is available for 2020 [here](https://www.bls.gov/lau/lastrk20.htm) and for 2022 [here](https://www.bls.gov/web/laus/laumstrk.htm). Iowa State University collected the BLS data from earlier years into a nice spreadsheet available [here](https://www.icip.iastate.edu/tables/employment/unemployment-states). 

In [229]:
unemployement_rates = pd.read_excel('emp-unemployment.ods', sheet_name='States')

In [230]:
unemployement_rates.head()

,State,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Alabama,8.9,10.6,14.1,13.8,11.0,9.2,9.7,8.1,7.2,7.0,6.8,7.3,7.6,7.3,6.2,6.0,5.2,5.0,4.4,4.7,4.6,5.1,5.9,6.0,5.7,4.5,4.0,4.0,5.7,11.0,10.5,9.6,8.0,7.2,6.8,6.1,5.8,4.4,3.9
1,Alaska,9.6,9.4,9.9,9.9,9.8,9.7,10.9,10.3,8.8,7.0,7.2,8.5,8.9,7.7,7.6,7.3,7.6,7.1,6.3,6.5,6.4,6.4,7.3,7.8,7.5,6.9,6.6,6.3,6.7,7.7,7.9,7.6,7.1,7.0,6.9,6.5,6.9,7.0,6.6
2,Arizona,6.6,6.2,10.1,8.8,5.2,6.3,6.9,6.5,6.4,5.3,5.3,5.9,7.5,6.4,6.1,5.3,5.6,4.6,4.3,4.4,4.0,4.8,6.1,5.7,5.0,4.7,4.2,3.9,6.2,9.9,10.4,9.5,8.3,7.7,6.8,6.1,5.4,4.9,4.8
3,Arkansas,7.6,8.7,9.9,9.9,8.7,8.7,8.6,8.1,7.6,7.0,6.9,7.4,7.1,6.1,5.4,4.8,5.3,5.2,5.2,4.6,4.3,5.0,5.5,5.9,5.7,5.2,5.2,5.3,5.5,7.8,8.2,8.3,7.6,7.2,6.0,5.0,4.0,3.7,3.7
4,California,6.8,7.4,10.0,9.8,7.8,7.2,6.7,5.8,5.3,5.1,5.8,7.7,9.3,9.5,8.6,7.9,7.3,6.4,5.9,5.2,4.9,5.4,6.7,6.8,6.2,5.4,4.9,5.4,7.3,11.2,12.2,11.7,10.4,8.9,7.5,6.2,5.5,4.8,4.2


In [231]:
unemp_2020 = pd.read_excel('emp-unemployment.ods', sheet_name='2020 rates')
unemp_2022 = pd.read_excel('emp-unemployment.ods', sheet_name='2022 rates')

In [232]:
unemployement_rates = unemployement_rates.merge(unemp_2020, on='State')
unemployement_rates = unemployement_rates.merge(unemp_2022, on='State')

In [233]:
unemployement_rates.head()

,State,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2020,2022
0,Alabama,8.9,10.6,14.1,13.8,11.0,9.2,9.7,8.1,7.2,7.0,6.8,7.3,7.6,7.3,6.2,6.0,5.2,5.0,4.4,4.7,4.6,5.1,5.9,6.0,5.7,4.5,4.0,4.0,5.7,11.0,10.5,9.6,8.0,7.2,6.8,6.1,5.8,4.4,3.9,6.5,2.6
1,Alaska,9.6,9.4,9.9,9.9,9.8,9.7,10.9,10.3,8.8,7.0,7.2,8.5,8.9,7.7,7.6,7.3,7.6,7.1,6.3,6.5,6.4,6.4,7.3,7.8,7.5,6.9,6.6,6.3,6.7,7.7,7.9,7.6,7.1,7.0,6.9,6.5,6.9,7.0,6.6,8.2,4.6
2,Arizona,6.6,6.2,10.1,8.8,5.2,6.3,6.9,6.5,6.4,5.3,5.3,5.9,7.5,6.4,6.1,5.3,5.6,4.6,4.3,4.4,4.0,4.8,6.1,5.7,5.0,4.7,4.2,3.9,6.2,9.9,10.4,9.5,8.3,7.7,6.8,6.1,5.4,4.9,4.8,7.7,3.5
3,Arkansas,7.6,8.7,9.9,9.9,8.7,8.7,8.6,8.1,7.6,7.0,6.9,7.4,7.1,6.1,5.4,4.8,5.3,5.2,5.2,4.6,4.3,5.0,5.5,5.9,5.7,5.2,5.2,5.3,5.5,7.8,8.2,8.3,7.6,7.2,6.0,5.0,4.0,3.7,3.7,6.1,3.4
4,California,6.8,7.4,10.0,9.8,7.8,7.2,6.7,5.8,5.3,5.1,5.8,7.7,9.3,9.5,8.6,7.9,7.3,6.4,5.9,5.2,4.9,5.4,6.7,6.8,6.2,5.4,4.9,5.4,7.3,11.2,12.2,11.7,10.4,8.9,7.5,6.2,5.5,4.8,4.2,10.2,4.1


In [234]:
def add_unemployment_rates(year):
    df = add_pres_approval(year)
    unemployment_df = unemployement_rates[['State', year]].copy()
    unemployment_df['State'] = unemployment_df['State'].str.upper()
    unemployment_df.columns = ['state', 'unemployment_rate']
    df = df.merge(unemployment_df, on='state')
    df['unemp_rate_int'] = df['dem_pres'] * df['unemployment_rate']
    return df

In [235]:
add_unemployment_rates(2012)

,state,year,partisan_score,old_score_avg,generic_ballot,pres_approval,dem_pres,pres_approval_int,unemployment_rate,unemp_rate_int
0,ARIZONA,2012,-0.030273,-0.170251,-0.002,0.065,1,0.065,8.3,8.3
1,CONNECTICUT,2012,0.124632,0.092673,-0.002,0.065,1,0.065,8.3,8.3
2,DELAWARE,2012,0.374657,0.354364,-0.002,0.065,1,0.065,7.2,7.2
3,FLORIDA,2012,0.130060,-0.008681,-0.002,0.065,1,0.065,8.5,8.5
4,HAWAII,2012,0.248182,0.389033,-0.002,0.065,1,0.065,6.0,6.0
5,INDIANA,2012,0.057638,-0.145798,-0.002,0.065,1,0.065,8.3,8.3
6,MAINE,2012,0.214125,-0.381991,-0.002,0.065,1,0.065,7.5,7.5
7,MARYLAND,2012,0.296494,0.182249,-0.002,0.065,1,0.065,7.0,7.0
8,MASSACHUSETTS,2012,0.074838,0.355538,-0.002,0.065,1,0.065,6.7,6.7
9,MICHIGAN,2012,0.208136,0.222356,-0.002,0.065,1,0.065,9.1,9.1


## Race and Ethnicity

For race and ethnicity data, I downloaded data from the CDC's Bridged-Race Population Estimates. The data is available [here](https://wonder.cdc.gov/controller/datarequest/D178).

In [269]:
race_data = pd.read_table('Bridged-Race Population Estimates 1990-2020.txt')

In [274]:
race_data.rename(columns={'Yearly July 1st Estimates': 'year'}, inplace=True)

In [275]:
race_data.head()

,Notes,Race,Race Code,State,State Code,year,Yearly July 1st Estimates Code,Population
0,NaN,American Indian or Alaska Native,1002-5,Alabama,1.0,1990.0,1990.0,16224.0
1,NaN,American Indian or Alaska Native,1002-5,Alabama,1.0,1991.0,1991.0,17015.0
2,NaN,American Indian or Alaska Native,1002-5,Alabama,1.0,1992.0,1992.0,17357.0
3,NaN,American Indian or Alaska Native,1002-5,Alabama,1.0,1993.0,1993.0,18002.0
4,NaN,American Indian or Alaska Native,1002-5,Alabama,1.0,1994.0,1994.0,18566.0


In [287]:
short_race_data = race_data[['State','year','Race','Population']].copy()

In [288]:
short_race_data.head()

,State,year,Race,Population
0,Alabama,1990.0,American Indian or Alaska Native,16224.0
1,Alabama,1991.0,American Indian or Alaska Native,17015.0
2,Alabama,1992.0,American Indian or Alaska Native,17357.0
3,Alabama,1993.0,American Indian or Alaska Native,18002.0
4,Alabama,1994.0,American Indian or Alaska Native,18566.0


In [289]:
short_race_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6560 entries, 0 to 6559
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   State       6528 non-null   object 
 1   year        6324 non-null   float64
 2   Race        6532 non-null   object 
 3   Population  6533 non-null   float64
dtypes: float64(2), object(2)
memory usage: 205.1+ KB


In [290]:
short_race_data.isnull().sum()

State          32
year          236
Race           28
Population     27
dtype: int64

In [291]:
short_race_data['year'].value_counts(dropna=False)

NaN       236
1991.0    204
2020.0    204
2019.0    204
2018.0    204
2017.0    204
2016.0    204
2015.0    204
2014.0    204
2013.0    204
2012.0    204
2011.0    204
2010.0    204
2009.0    204
2008.0    204
2007.0    204
1990.0    204
2005.0    204
2004.0    204
2003.0    204
2002.0    204
2001.0    204
2000.0    204
1999.0    204
1998.0    204
1997.0    204
1996.0    204
1995.0    204
1994.0    204
1993.0    204
1992.0    204
2006.0    204
Name: year, dtype: int64

In [292]:
short_race_data.dropna(inplace=True)

In [293]:
short_race_data['year'] = short_race_data['year'].astype(int)

In [294]:
short_race_data.head()

,State,year,Race,Population
0,Alabama,1990,American Indian or Alaska Native,16224.0
1,Alabama,1991,American Indian or Alaska Native,17015.0
2,Alabama,1992,American Indian or Alaska Native,17357.0
3,Alabama,1993,American Indian or Alaska Native,18002.0
4,Alabama,1994,American Indian or Alaska Native,18566.0


In [328]:
def add_race(year):
    df = add_unemployment_rates(year)
    race_year_data = short_race_data[short_race_data['year'] == year]
    race_year_data = race_year_data[['State','Race','Population']].pivot_table(index='State', columns=['Race'])
    race_year_data = race_year_data.reset_index(level=0)
    race_year_data.columns = ['state','native_amer','asian', 'black','white']
    race_year_data['state'] = race_year_data['state'].str.upper()
    race_year_data['tot_pop'] = race_year_data[['native_amer','asian', 'black','white']].sum(axis=1)
    for race in ['native_amer','asian', 'black','white']:
        race_year_data['{}_perc'.format(race)] = race_year_data[race] / race_year_data['tot_pop']
    #Below, white is ommitted to avoid perfect multicollinearity. 
    races_to_merge = race_year_data[['state','native_amer_perc','asian_perc', 'black_perc']]
    df = df.merge(races_to_merge, on='state')
    return df
    

In [329]:
add_race(2018).head()

,state,year,partisan_score,old_score_avg,generic_ballot,pres_approval,dem_pres,pres_approval_int,unemployment_rate,unemp_rate_int,native_amer_perc,asian_perc,black_perc
0,ARIZONA,2018,0.023445,-0.079944,0.073,-0.115,0,-0.0,4.8,0.0,0.057111,0.043274,0.058406
1,CONNECTICUT,2018,0.174474,0.177630,0.073,-0.115,0,-0.0,4.1,0.0,0.007079,0.052338,0.129221
2,DELAWARE,2018,0.221345,0.255334,0.073,-0.115,0,-0.0,3.8,0.0,0.007563,0.044511,0.241947
3,FLORIDA,2018,-0.001225,0.026699,0.073,-0.115,0,-0.0,3.6,0.0,0.005679,0.033618,0.175966
4,HAWAII,2018,0.423022,0.368515,0.073,-0.115,0,-0.0,2.4,0.0,0.004503,0.662591,0.030587


## Combining all the Data!

Now, I will combine the data from multiple election years to form the data set I will use to train and test our model. For now, I will combine the data for elections from 2008-2020.

In [345]:
def combine_years(year_list):
    final_data = add_race(year_list[0])
    for year in year_list[1:]:
        final_data = pd.concat([final_data, add_race(year)])
    return final_data

In [346]:
combined_elections = combine_years([2008, 2010, 2012, 2014, 2016, 2018, 2020])

In [347]:
combined_elections

,state,year,partisan_score,old_score_avg,generic_ballot,pres_approval,dem_pres,pres_approval_int,unemployment_rate,unemp_rate_int,native_amer_perc,asian_perc,black_perc
0,ALABAMA,2008,-0.268418,-0.269701,0.100,-0.422,0,-0.0,5.7,0.0,0.007104,0.012499,0.266876
1,ALASKA,2008,0.012442,-0.030323,0.100,-0.422,0,-0.0,6.7,0.0,0.170740,0.066970,0.043748
2,COLORADO,2008,0.103035,-0.000784,0.100,-0.422,0,-0.0,4.8,0.0,0.017203,0.032577,0.047404
3,DELAWARE,2008,0.293730,0.294580,0.100,-0.422,0,-0.0,4.9,0.0,0.006676,0.033771,0.223802
4,GEORGIA,2008,-0.089013,-0.123685,0.100,-0.422,0,-0.0,6.2,0.0,0.005107,0.034483,0.311697
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,SOUTH DAKOTA,2020,-0.314705,-0.322578,0.068,-0.090,0,-0.0,4.3,0.0,0.095687,0.018982,0.029533
26,TENNESSEE,2020,-0.270371,-0.203942,0.068,-0.090,0,-0.0,7.4,0.0,0.005309,0.022631,0.177039
27,TEXAS,2020,-0.096394,-0.148814,0.068,-0.090,0,-0.0,7.7,0.0,0.011580,0.058641,0.135696
28,VIRGINIA,2020,0.120838,0.084033,0.068,-0.090,0,-0.0,6.2,0.0,0.006291,0.078151,0.209706


In [348]:
# combined_elections.to_csv('combined_election_data.csv')